# Tutorial for MIRI Coronagraphy Reduction with spaceKLIP

---

In this notebook, you will learn how to reduce MIRI coronagraphy data from the JWST ERS program on Direct Observations of Exoplanetary Systems ([Program 1386](https://www.stsci.edu/jwst/science-execution/program-information?id=1386)), with a focus on the exoplanet HIP 65426 b. This tutorial guides you step-by-step through the data reduction process using the spaceKLIP pipeline, offering a clear and concise workflow tailored for effective high-contrast imaging analysis. By the end of this notebook, you will have gained hands-on experience with the tools and techniques necessary for reducing MIRI coronagraphic data, preparing you to apply these methods to other similar datasets.


<div class="alert alert-warning">
    
**Related Tutorials and Further Information**: This notebook is intentionally very similar to the ["Tutorial for NIRCam Coronagraphy Reduction with spaceKLIP"](https://spaceklip.readthedocs.io/en/latest/tutorials/tutorial_NIRCam_reductions.html) notebook. Subsequent analyses will be done in the ["Tutorial for MIRI Post-Pipeline Contrast Analyses Using spaceKLIP"](https://spaceklip.readthedocs.io/en/latest/tutorials/tutorial_MIRI_contrast_analyses.html) notebook. For complete interactive plotting capabilities, download the notebook and execute it locally.
</div>

<div class="alert alert-info">
    
**MIRI-specific Information**: Steps and information specific to MIRI are called out in blue.</div>

<h1 style="font-size: 24px;">Table of Contents</h1>

* [1. Introduction](#Introduction)
* [2. Setup and Imports](#Setup-and-Imports)
* [3. Download the Data](#Precursor:-Download-the-Data)
* [4. Stage 1 Reductions](#Stage-1-Reductions)
    * [4.1 Index Files into Database for Stage 1](#Index-Files-into-Database-for-Stage-1)
    * [4.2 Run Stage 1 Pipeline](#Run-Stage-1-Pipeline)
    * [4.3 Display Stage 1 Reductions](#Display-Stage-1-Reductions)
* [5. Stage 2 Reductions](#Stage-2-Reductions)
    * [5.1 Re-read Stage 1 Outputs into Database](#Optional:-Re-read-Stage-1-Outputs-into-Database)
    * [5.2 Run Stage 2 Pipeline](#Run-Stage-2-Pipeline)
    * [5.3 Display Stage 2 Reductions](#Display-Stage-2-Reductions)
* [6. Stage 3 reductions: Preparations for PSF subtraction](#Stage-3-Reductions:-Preparations-for-PSF-Subtraction
)
    * [6.1 Re-read Stage 2 Outputs into Database](#Optional:-Re-read-Stage-2-Outputs-into-Database)
    * [6.2 Using spaceKLIP ImageTools](#Using-spaceKLIP-ImageTools)
    * [6.3 Crop the Frames](#Crop-the-Frames)  
    * [6.4 Find and Repair Bad Pixels](#Find-and-Repair-Bad-Pixels)
    * [6.5 Remove Background](#Remove-Background)
    * [6.5 Finish Pixel Cleanup](#Finish-Pixel-Cleanup)
    * [6.6 Align Frames](#Align-Frames)
    * [6.7 Pad Empty Space Around Frames](#Pad-Empty-Space-Around-Frames)
    * [6.8 Display the Cleaned Datasets](#Display-the-Cleaned-Datasets)
* [7. Stage 3 Reductions: KLIP](#Stage-3-Reductions:-KLIP)
    * [7.1 PSF Subtraction: Option Using pyKLIP](#PSF-Subtraction:-Option-Using-pyKLIP)
    * [7.2 Re-read Stage 3 Outputs into Database](#Optional:-Re-read-Stage-3-Outputs-into-Database)

---

## Introduction

A comprehensive introduction to High-Contrast Imaging (HCI) techniques and challenges, including differential imaging methods and point spread function (PSF) subtraction algorithms, can be found in the ["Tutorial for NIRCam Coronagraphy Reduction with spaceKLIP"](https://spaceklip.readthedocs.io/en/latest/tutorials/tutorial_NIRCam_reductions.html) notebook. In this notebook, we introduce the coronagraphic capabilities of the Mid-Infrared Instrument (MIRI) aboard the James Webb Space Telescope (JWST).

MIRI offers coronagraphic imaging with four individual coronagraphs—one [Lyot-type coronagraph](https://jwst-docs.stsci.edu/jwst-mid-infrared-instrument/miri-instrumentation/miri-coronagraphs#MIRICoronagraphs-lyot&gsc.tab=0:~:text=al.%20(2022).-,Lyot%20coronagraph,-The%20Lyot%20coronagraph) and three [4-quadrant phase-mask (4QPM) coronagraphs](https://jwst-docs.stsci.edu/jwst-mid-infrared-instrument/miri-instrumentation/miri-coronagraphs#MIRICoronagraphs-lyot&gsc.tab=0:~:text=4%2Dquadrant%20phase%20mask%20(4QPMs)%20coronagraphs%C2%A0)—that facilitate HCI at wavelengths of 10–23 µm.

* **Four-Quadrant Phase Mask (FQPM) Coronagraph**: Unlike the occulting mask used in a Lyot-type coronagraph, an FQPM is a four-quant transparent mask centered on the star. The FQPM introduces a 180° phase shift in the light transmitted through two diagonal quadrants. This phase manipulation results in destructive interference at the star’s position, effectively canceling its light and allowing off-axis sources (companions) to pass through with minimal distortion. MIRI’s FQPM coronagraphs provide the smallest possible inner working angle (IWA) of ~1 λ/D at 10-16 μm, making it particularly effective for detecting objects very close to the star. However, each operates over a narrow wavelength range.
  
* **Lyot Coronagraph**: MIRI and NIRCam Lyot-type coronagraphs have similar designs, featuring two binary masks: the occulter and the Lyot stop. The occulter is positioned at the initial focal plane and blocks light from the central star, allowing light from companions to pass through. The Lyot stop in the re-imaged pupil plane blocks residual diffracted light from the star. The occulter radius defines the IWA, MIRI’s being ~3.3 λ/D at 23 µm, and is particularly useful for investigating structures and diffuse emissions near bright sources, such as protoplanetary and debris disks.


While coronagraphs block most of the starlight, there is some star leakage (or diffracted starlight) that propagates to the detector and results in a residual starlight "speckle" pattern or static wavefront errors that require additional post-processing and imaging techniques to remove, which we will explore in the remainder of this notebook using the [spaceKLIP data reduction pipeline](https://spaceklip.readthedocs.io/en/latest/).


---

## Setup and Imports

In [1]:
import os
import numpy as np
import subprocess

import spaceKLIP

import matplotlib
matplotlib.rcParams.update({'font.size': 14})
%matplotlib inline

Note that currently the import of `webbpsf_ext` has a side effect of configuring extra verbose logging. We're not interested in that logging text, so let's quiet it. 


In [2]:
import webbpsf_ext
webbpsf_ext.setup_logging('WARN', verbose=False)

---

## Precursor: Download the Data

If you already have a copy of this data, you can adjust the paths below accordingly. In this notebook, we assume you don't have the data yet, so let's download it here.

We will use the `jwst_mast_query` [package](https://github.com/spacetelescope/jwst_mast_query) for this. Consult the package's documentation for more details.

We'll download all the uncalibrated raw data (`uncal.fits`), as we will use spaceKLIP to invoke the [JWST pipeline](https://github.com/spacetelescope/jwst) with customized options and extra steps optimized for coronagraphy.

In [3]:
data_root = 'data_miri_hd65426'

In [4]:
# Make subdirectories to put the data in.

os.makedirs(data_root, exist_ok=True)
os.makedirs(os.path.join(data_root, 'uncal'), exist_ok=True)

# Invoke the download.
download_cmd = (
    "yes | jwst_download.py --propID 1386 -i miri -l 1200 "
    "--obsnums 4 5 6 7 8 9 28 29 30 31 "
    "--outsubdir data_miri_hd65426/uncal --skip_propID2outsubdir "
    "-f uncal --date_select 59371.0+"
)

process=subprocess.Popen(download_cmd, shell=True,
                         stdout=subprocess.PIPE,
                         stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

# Uncomment to print the download log and any errors.
# print(stdout.decode())
# print(stderr.decode())

---

## Stage 1 Reductions

### Index Files into Database for Stage 1
SpaceKLIP relies on a `Database` class to track observations, data files, and the relationships between them. 

We begin by creating a database and reading files into it. 

For this tutorial, let's only reduce one filter's worth of data. 

In [5]:
filt = 'F1550C'  # Set to None to disable filter selection and load all filters.

In [6]:
# Initialize spaceKLIP database.
database = spaceKLIP.database.create_database(input_dir=os.path.join(data_root, 'uncal'),
                                              output_dir=data_root,
                                              assoc_using_targname=False,
                                              filt=filt,
                                              pid=1386)

[spaceKLIP.database:INFO] --> Identified 1 concatenation(s)
[spaceKLIP.database:INFO]   --> Concatenation 1: JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
 TYPE  EXP_TYPE DATAMODL TELESCOP ... PIXAR_SR BUNIT      ROLL_REF      BLURFWHM
------ -------- -------- -------- ... -------- ----- ------------------ --------
   SCI MIR_4QPM   STAGE0     JWST ...      nan    DN 108.17111983245404      nan
   SCI MIR_4QPM   STAGE0     JWST ...      nan    DN 117.54987336203172      nan
SCI_BG MIR_4QPM   STAGE0     JWST ...      nan    DN 112.74290651328519      nan
SCI_BG MIR_4QPM   STAGE0     JWST ...      nan    DN  112.7429014011905      nan
REF_BG MIR_4QPM   STAGE0     JWST ...      nan    DN 112.79990673202992      nan
REF_BG MIR_4QPM   STAGE0     JWST ...      nan    DN  112.7999802706289      nan
   REF MIR_4QPM   STAGE0     JWST ...      nan    DN 109.21467777818188      nan
   REF MIR_4QPM   STAGE0     JWST ...      nan    DN 109.21466715208938      nan
   REF MIR_4QPM   STAGE0     JW

The above is a bit verbose and can be difficult for a human to parse; let's ask the database to summarize what it contains:

In [7]:
database.summarize()

MIRI_F1550C_4QPM
	STAGE0: 15 files;	2 SCI, 9 REF, 4 BG


Above, you should notice three types of files contained in the database:

* **Science** (`SCI`): These files hold the primary coronagraphic observational data of the target—in this case, the exoplanet host star HIP 65426. This program also conducted coronagraphic observations at two separate roll angles, which refer to specific pointing/orientation of the telescope. Hence, there are two `SCI` files.
* **Reference** `REF`: These files contain the reference PSF observations of other stars—in this case, of the nearby star HIP 68245. There are nine `REF` files, one for each dithered exposure.
* **Background** (`SCI_BG` and `REF_BG`): These files contain the background data for each roll, corresponding to both the science and reference targets.

---

### Run Stage 1 Pipeline

The `Coron1Pipeline` (`calwebb_coron1`[↘️](https://spaceklip.readthedocs.io/en/latest/stage1.html)) in spaceKLIP is a custom subclass of the JWST Stage 1 pipeline, `Detector1Pipeline` (`calwebb_detector1`[↘️](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#calwebb-detector1)). It is specifically designed to optimize the processing of high-contrast imaging data typical of coronagraphic observations. This pipeline applies group-by-group detector-level corrections, followed by ramp fitting, to the raw, uncalibrated data (`uncal.fits`[↘️](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#:~:text=g.%20%E2%80%9Cjw80600012001_02101_00003_mirimage_ramp.fits%E2%80%9D.-,Inputs,-%EF%83%81)). The output is calibrated count rate products (`rateints.fits`[↘️](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#:~:text=be%20ImageModel.-,3D%20countrate%20product,%EF%83%81,-Data%20model)).

The `Coron1Pipeline` performs the following steps for MIRI. Custom spaceKLIP steps and parameters are marked with stars (⭐). Click on the steps with the attached (ReadtheDocs 📄) links to learn more about specific JWST pipeline steps.


> * `group_scale` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/group_scale/description.html))</span>
> * `dq_init`  <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/dq_init/description.html))</span>
> * `saturation` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/saturation/description.html))</span>    
>    *  `flag_rcsat`: Flag high dark current RC pixels as saturated? (Default: False).    
>    *  `grow_diagonal`: Flag diagonal neighboring pixels (or only bottom/top/left/right)? (Default: True).
>    * `n_pix_grow_sat` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/saturation/arguments.html))</span>
> * `ipc` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/ipc/description.html))</span>
> * `firstframe` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/firstframe/description.html))</span>
> * `lastframe` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/lastframe/description.html))</span>
> * `reset` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/reset/description.html))</span>
> * `linearity` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/linearity/description.html))</span>     
> * `rscd` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/rscd/description.html))</span>
> * `dark_current` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/dark_current/description.html))</span>
> * `refpix` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/refpix/index.html))</span>
>    * `odd_even_columns` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/refpix/arguments.html#:~:text=%2D%2D-,odd_even_columns,-If%20the%20odd_even_columns))</span>
>    * ⭐ `nlower`, `nupper`, `nleft`, `nright`: Number of pixels around edges to use as pseudo-refpix in NIRCam subarrays (Default: 4).   
>    * ⭐ `nrow_off`, `ncol_off`: Offset the reference pixel region from the top/bottom and left/right edges of the frame (Default: 0).
> * `jump` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/jump/description.html))</span>
>    * `rejection_threshold` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/jump/arguments.html#:~:text=%2D%2D-,rejection_threshold,-%3A%20A%20floating%2Dpoint))</span> , `three_group_rejection_threshold` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/jump/arguments.html#:~:text=three_group_rejection_threshold))</span>,
>    * `four_group_rejection_threshold` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/jump/arguments.html#:~:text=four_group_rejection_threshold))</span> , `maximum_cores` <span style="font-size: 8px;">([ReadtheDocs📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/jump/arguments.html#:~:text=%2D%2D-,maximum_cores,-%3A%20The%20number%20of))</span>
> * `ramp_fit` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/ramp_fitting/description.html))</span>
>    * ⭐ `save_calibrated_ramp`: Save the calibrated ramp? The default is False.
> * `gain_scale` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/gain_scale/description.html))</span>


The following cell will run the `Coron1Pipeline` for all input data in the spaceKLIP database, saving the output to a subdirectory named `stage1`. This can take a long time to run, so be patient. 

In [ ]:
spaceKLIP.coron1pipeline.run_obs(database=database,
                       steps={'group_scale': {'skip': False},
                              'dq_init': {'skip': False, 'save_results':True},
                              'saturation': {'n_pix_grow_sat': 1,
                                             'grow_diagonal': False},
                              'ipc': {'skip': True},
                              'firstframe': {'skip': False},
                              'lastframe': {'skip': False},
                              'reset': {'skip': False},
                              'linearity': {'skip': False},
                              'rscd': {'skip': False},
                              'dark_current': {'skip': True},
                              'refpix': {'skip': False,
                                         'odd_even_columns': True,
                                         'odd_even_rows': True,
                                         'nlower': 0,
                                         'nupper': 0,
                                         'nleft': 0,
                                         'nright': 0,
                                         'nrow_off': 0,
                                         'ncol_off': 0},
                              'jump': {'rejection_threshold': 8.,
                                       'three_group_rejection_threshold': 8.,
                                       'four_group_rejection_threshold': 8.,
                                       'maximum_cores': 'all'},
                              'ramp_fit': {'save_calibrated_ramp': False,
                                           'maximum_cores': 'all'},
                              'gain_scale': {'skip': False}},
                       subdir='stage1')

[spaceKLIP.coron1pipeline:INFO] --> Concatenation JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
[spaceKLIP.coron1pipeline:INFO]   --> Coron1Pipeline: processing jw01386008001_04101_00001_mirimage_uncal.fits


We can now examine the updated database, which shows that all available files for each filter have been processed to Stage 1.

**Note**: The Stage 0 files are automatically removed from the database since there is no further processing required for them. However, the files remain on disk.


In [9]:
database.summarize()

MIRI_F1550C_4QPM
	STAGE1: 15 files;	2 SCI, 9 REF, 4 BG


---

### Display Stage 1 Reductions

Let's examine the science and reference PSF data in the F1550C filter we processed through the `Coron1Pipeline`. You can use the built-in plotting function `spaceKLIP.plotting.display_coron_dataset` to display the images by passing the database object to the function. Each image includes annotations, with pixels marked as `DO_NOT_USE` in the data quality (DQ) extension highlighted in orange. Additionally, the plotting function allows you to restrict the display to filter specific data and save the images. The `restrict_to` parameter can be a simple string that filters by matching keys in the database or a dictionary that applies filters based on specific columns in the database table. The images will be saved as a PDF file to the current working directory by default or to a specified path if provided by the user passed to the `save_filename` keyword argument. There are also additional parameters, such as `vmin`, `vmax`, and `stretch`, that allow you to adjust the visualization settings for image normalization.

To browse through the files in the database interactively, set `interactive=True`. Doing this will enable a slider to flip through the images. To generate and save static plots in a PDF, set `interactive=False`.


In [9]:
spaceKLIP.plotting.display_coron_dataset(
    database,
    restrict_to={
        'FILTER': filt,  # Sort by filter.
        'TYPE': ['SCI']  # Sort by file type SCI/REF.
    },
    interactive=True,  # Static or interactive plots?
    zoom_center=3,  # Optional zoom factor; set to None to disable.
    # vmin=3, vmax=500,  # Define the min/max values for consistent image scaling.
    save_filename='plots_f1550c_stage1.pdf'  # Save plots to PDF.
)

IntSlider(value=0, description='Image  Index:', max=1)

Output()

**Note:** The leftmost four columns are reference pixels. The adjacent orange columns contain a mix of `NON_SCIENCE` pixels and those flagged as `DO_NOT_USE` by the MIRI bad pixel mask during the `dq_init` step. Before PSF subtraction, we will crop these images to retain only the coronagraphic data. Also the PSF may be difficult to see at this stage, but will become more visible after the background is subtracted in a later step.

---

## Stage 2 Reductions

### Optional: Re-read Stage 1 Outputs into Database 

This shows how you can start re-reductions at this stage in the processing, once the previous steps have been completed. You might want to re-read the data if, for example, you have been provided with files that have already been processed through Stage 1 but require further reductions.

In [10]:
database = spaceKLIP.database.create_database(
                                    input_dir=os.path.join(data_root, 'stage1'),
                                    file_type='rateints.fits',
                                    output_dir=data_root,
                                    assoc_using_targname=False,
                                    filt=filt,
                                    pid=1386)

[spaceKLIP.database:INFO] --> Identified 1 concatenation(s)
[spaceKLIP.database:INFO]   --> Concatenation 1: JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
 TYPE  EXP_TYPE DATAMODL TELESCOP ... PIXAR_SR BUNIT      ROLL_REF      BLURFWHM
------ -------- -------- -------- ... -------- ----- ------------------ --------
   SCI MIR_4QPM   STAGE1     JWST ...      nan  DN/s 108.17111983245404      nan
   SCI MIR_4QPM   STAGE1     JWST ...      nan  DN/s 117.54987336203172      nan
SCI_BG MIR_4QPM   STAGE1     JWST ...      nan  DN/s 112.74290651328519      nan
SCI_BG MIR_4QPM   STAGE1     JWST ...      nan  DN/s  112.7429014011905      nan
REF_BG MIR_4QPM   STAGE1     JWST ...      nan  DN/s 112.79990673202992      nan
REF_BG MIR_4QPM   STAGE1     JWST ...      nan  DN/s  112.7999802706289      nan
   REF MIR_4QPM   STAGE1     JWST ...      nan  DN/s 109.21467777818188      nan
   REF MIR_4QPM   STAGE1     JWST ...      nan  DN/s 109.21466715208938      nan
   REF MIR_4QPM   STAGE1     JW

---

### Run Stage 2 Pipeline

The `Coron2Pipeline` (`calwebb_coron2`[↘️](https://spaceklip.readthedocs.io/en/latest/stage2.html)) in spaceKLIP is a customized subclass of the JWST Stage 2 Imaging Pipeline, `Image2Pipeline` (`calwebb_image2`[↘️](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html)), specifically designed to optimize the processing of high-contrast imaging data typical of coronagraphic observations. This stage requires little customization. This pipeline performs additional corrections and calibrations on the countrate products (`rateints.fits`[↘️](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html#:~:text=per%2Dintegration%20results)) from stage 1 to produce fully calibrated products (`calints.fits`[↘️](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html#:~:text=2D%20or%203D%20calibrated,%EF%83%81)). 

The `Coron2Pipeline` includes the following steps for MIRI. Custom spaceKLIP steps and parameters are marked with stars (⭐). Click on the steps with the attached (ReadtheDocs 📄) links to learn more about specific JWST pipeline steps.

> * `bkg_subtract` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/background_step/index.html#background-step))</span>
> * `assign_wcs` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/assign_wcs/index.html#assign-wcs-step))</span>
> * `flat_field` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/flatfield/index.html#flatfield-step))</span>
> * `photom` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/photom/index.html#photom-step))</span>
> * ⭐ `outlier_detection` <span style="font-size: 8px;">([ReadtheDocs 📄](https://jwst-pipeline.readthedocs.io/en/latest/jwst/outlier_detection/outlier_detection_coron.html))</span>

The following cell will run the `Coron2Pipeline` for all input data in the spaceKLIP database, saving the output to a subdirectory named `stage2`.

In [11]:
spaceKLIP.coron2pipeline.run_obs(database=database,
                           steps={'bkg_subtract': {'skip': False},
                                  'assign_wcs': {'skip': False},
                                  'flat_field': {'skip': False},
                                  'photom': {'skip': False},
                                  'outlier_detection': {'skip': False}},
                           subdir='stage2')

[spaceKLIP.coron2pipeline:INFO] --> Concatenation JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
[spaceKLIP.coron2pipeline:INFO]   --> Coron2Pipeline: processing jw01386008001_04101_00001_mirimage_rateints.fits
[spaceKLIP.coron2pipeline:INFO]   --> Coron2Pipeline: processing jw01386009001_04101_00001_mirimage_rateints.fits
[spaceKLIP.coron2pipeline:INFO]   --> Coron2Pipeline: processing jw01386030001_02101_00001_mirimage_rateints.fits
[spaceKLIP.coron2pipeline:INFO]   --> Coron2Pipeline: processing jw01386030001_03101_00001_mirimage_rateints.fits
[spaceKLIP.coron2pipeline:INFO]   --> Coron2Pipeline: processing jw01386031001_02101_00001_mirimage_rateints.fits
[spaceKLIP.coron2pipeline:INFO]   --> Coron2Pipeline: processing jw01386031001_03101_00001_mirimage_rateints.fits
[spaceKLIP.coron2pipeline:INFO]   --> Coron2Pipeline: processing jw01386007001_04101_00001_mirimage_rateints.fits
[spaceKLIP.coron2pipeline:INFO]   --> Coron2Pipeline: processing jw01386007001_04101_00002_mirimage_ra

Again, we can check that the database now contains stage 2 reduced versions of all the files:

In [12]:
database.summarize()

MIRI_F1550C_4QPM
	STAGE2: 15 files;	2 SCI, 9 REF, 4 BG


---

### Display Stage 2 Reductions

These images look nearly identical to the Stage 1 outputs, but note that the display units have been rescaled from DN/s (countrate) to physical units of MJy/sr (surface brightness). You may also notice that more pixels have been flagged as `DO_NOT_USE` after applying the outlier detection step in Stage 2.


In [13]:
spaceKLIP.plotting.display_coron_dataset(
    database,
    restrict_to={
        'FILTER': filt,  # Sort by filter.
        'TYPE': ['SCI']  # Sort by file type SCI/REF.
    },
    interactive=True,  # Static or interactive plots?
    zoom_center=3,  # Optional zoom factor; set to None to disable.
    # vmin=3, vmax=1e3,  # Define the min/max values for consistent image scaling.
    save_filename='plots_f1550c_stage2.pdf'  # Save plots to PDF.
)

IntSlider(value=0, description='Image  Index:', max=1)

Output()

---

## Stage 3 Reductions: Preparations for PSF Subtraction

As is often the case in high-contrast imaging, obtaining good PSF subtractions depends sensitively on preparing the data ahead of time.

In the following section, we improve coronagraphic reductions, taking special care with image centering, background subtraction, and bad pixel replacement/interpolation, all before the PSF subtraction steps.

---

### Optional: Re-read Stage 2 Outputs into Database
This shows how you can start re-reductions at this stage in the processing, once the previous steps have been completed.

In [14]:
database = spaceKLIP.database.create_database(
                                    input_dir=os.path.join(data_root, 'stage2'),
                                    file_type='calints.fits',
                                    output_dir=data_root,
                                    assoc_using_targname=False,
                                    filt=filt,
                                    pid=1386)


[spaceKLIP.database:INFO] --> Identified 1 concatenation(s)
[spaceKLIP.database:INFO]   --> Concatenation 1: JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
 TYPE  EXP_TYPE DATAMODL TELESCOP ... BUNIT       ROLL_REF      BLURFWHM
------ -------- -------- -------- ... ------ ------------------ --------
   SCI MIR_4QPM   STAGE2     JWST ... MJy/sr 108.17111983245404      nan
   SCI MIR_4QPM   STAGE2     JWST ... MJy/sr 117.54987336203172      nan
SCI_BG MIR_4QPM   STAGE2     JWST ... MJy/sr 112.74290651328519      nan
SCI_BG MIR_4QPM   STAGE2     JWST ... MJy/sr  112.7429014011905      nan
REF_BG MIR_4QPM   STAGE2     JWST ... MJy/sr 112.79990673202992      nan
REF_BG MIR_4QPM   STAGE2     JWST ... MJy/sr  112.7999802706289      nan
   REF MIR_4QPM   STAGE2     JWST ... MJy/sr 109.21467777818188      nan
   REF MIR_4QPM   STAGE2     JWST ... MJy/sr 109.21466715208938      nan
   REF MIR_4QPM   STAGE2     JWST ... MJy/sr 109.21467183279624      nan
   REF MIR_4QPM   STAGE2     JWST ... 

---

### Using spaceKLIP ImageTools

This is where we will do some extra image processing to improve coronagraphic reductions. SpaceKLIP's image manipulation tools class, `ImageTools`, allows you to perform tasks like image alignment, bad pixel cleaning, and more directly on the data in the database.


In [15]:
# Initialize spaceKLIP image manipulation tools class.
imageTools = spaceKLIP.imagetools.ImageTools(database=database)

---

### Crop the Frames

<div class="alert alert-info">
    
**MIRI-specific Information**: This step is required only for MIRI data. </div>

As shown in the images above, the MIRI data contains regions outside the coronagraph that are not needed. To focus on the relevant coronagraphic data, we will crop the images accordingly. The `npix` parameter determines how many pixels are cropped from the edges of the frames.

In [16]:
# Crop all frames.
imageTools.crop_frames(npix=[13, 60, 7, 7],  # [left, right, bottom, top]
                       types=['SCI', 'SCI_BG', 'REF', 'REF_BG'],
                       subdir='cropped')

[spaceKLIP.imagetools:INFO] --> Concatenation JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
[spaceKLIP.imagetools:INFO]   --> Frame cropping: jw01386008001_04101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Frame cropping: old shape = (224, 288), new shape = (210, 215)
[spaceKLIP.imagetools:INFO]   --> Frame cropping: jw01386009001_04101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Frame cropping: old shape = (224, 288), new shape = (210, 215)
[spaceKLIP.imagetools:INFO]   --> Frame cropping: jw01386030001_02101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Frame cropping: old shape = (224, 288), new shape = (210, 215)
[spaceKLIP.imagetools:INFO]   --> Frame cropping: jw01386030001_03101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Frame cropping: old shape = (224, 288), new shape = (210, 215)
[spaceKLIP.imagetools:INFO]   --> Frame cropping: jw01386031001_02101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO

Let's verify the cropped images.

In [17]:
spaceKLIP.plotting.display_coron_dataset(
    database,
    restrict_to={
        'FILTER': filt,  # Sort by filter.
        'TYPE': ['SCI']  # Sort by file type SCI/REF.
    },
    interactive=True,  # Static or interactive plots?
    zoom_center=3,  # Optional zoom factor; set to None to disable.
    # vmin=3, vmax=1e3,  # Define the min/max values for consistent image scaling.
    save_filename='plots_f1550c_cropped.pdf'  # Save plots to PDF.
)

IntSlider(value=0, description='Image  Index:', max=1)

Output()

---

### Find and Repair Bad Pixels

For MIRI, the JWST pipeline does not sufficiently repair bad pixels (i.e., anomalous outliers) within the coronagraphic subarrays.

Here, we use the custom functions within spaceKLIP to detect and repair these bad pixels.

<div class="alert alert-info">
    
**MIRI-specific Information**: The `fix_bad_pixels` step was originally designed for NIRCam and is optimized for NIRCam data. As a result, users may need to provide a custom bad pixel mask when using the spaceKLIP pipeline for MIRI data. </div>

<div class="alert alert-warning">
    
**WARNING**: The `fix_bad_pixels` function will soon be deprecated in spaceKLIP and replaced by the `find_bad_pixels` and `clean_bad_pixels` functions. The following cell uses the original approach for finding and cleaning bad pixels until the new methods are fully integrated.</div>


Different finding/cleaning methods should be combined in a single string with a '+' sign, without whitespace, to apply them sequentially.


In [18]:
# Fix bad pixels using custom spaceKLIP routines. Multiple routines can be
# combined in a custom order by joining them with a + sign.
# - sigclip: use sigma clipping to find additional bad pixels.
# - custom: use custom map to find additional bad pixels.
# - timemed: replace pixels which are only bad in some frames with their
#            median value from the good frames.
# - localmed: replace bad pixels with the median of surrounding good
#            pixels.
# - medfilt: replace bad pixels with an image plane median filter.

# - shift_x/y: Define the range of pixel shifts (left/right and up/down)
# used in sigma clipping to compare a pixel with its neighbors and identify deviations.

imageTools.fix_bad_pixels(method='sigclip+timemed+localmed+medfilt',
                          sigclip_kwargs={'sigclip': 3,
                                          'shift_x': [-2, -1, 0, 1, 2],
                                          'shift_y': [-2, -1, 0, 1, 2]},
                          custom_kwargs={},
                          timemed_kwargs={},
                          localmed_kwargs={'shift_x': [-1, 0, 1],
                                           'shift_y': [-1, 0, 1]},
                          medfilt_kwargs={'size': 4},
                          subdir='bpcleaned')

[spaceKLIP.imagetools:INFO] --> WARNING! The fix_bad_pixels() routine is deprecated, the ..........
[spaceKLIP.imagetools:INFO] --> WARNING! find_bad_pixels() and clean_bad_pixels() are preferred!!!!
[spaceKLIP.imagetools:INFO] --> Concatenation JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
[spaceKLIP.imagetools:INFO]   --> Method sigclip: jw01386008001_04101_00001_mirimage_calints.fits
Frame 60/60, iteration 3
[spaceKLIP.imagetools:INFO]   --> Method sigclip: identified 222302 additional bad pixel(s) -- 8.21%
[spaceKLIP.imagetools:INFO]   --> Method timemed: jw01386008001_04101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Method timemed: fixing 216002 bad pixel(s) -- 7.97%
[spaceKLIP.imagetools:INFO]   --> Method localmed: jw01386008001_04101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Method localmed: fixing 27660 bad pixel(s) -- 1.02%
[spaceKLIP.imagetools:INFO]   --> Method medfilt: jw01386008001_04101_00001_mirimage_calints.fits
[spaceKLIP.im

Again, let's examine the results, but this time focusing on the cleaned products.


In [19]:
# Compare how well each cleaning method did to replace bad pixels.

spaceKLIP.plotting.display_image_comparisons(
    database,
    ['cropped', 'bpcleaned'],  # Subdirectories to look for files in.
    restrict_to={'FILTER': filt,  # Sort by filter.
                 'TYPE': ['SCI']  # Sort by file type SCI/REF.
    },
    interactive=True,  # Static or interactive plots?
    # vmin=0, vmax=1e3, # Define the min/max values for consistent image scaling.
    save_filename='clean_bp_f1550c_comparison.pdf')

IntSlider(value=0, description='Image Index:', max=1)

Output()

---

### Remove Background

<div class="alert alert-info">
    
**MIRI-specific Information**: MIRI flight data has revealed light scattering into the coronagraphs, creating linear features along the boundaries of the 4QPM, around the Lyot occulting spot, and near the bottom of the coronagraphic fields. The horizontal glow along the phase mask boundaries, referred to as ["glow sticks"](https://jwst-docs.stsci.edu/depreciated-jdox-articles/miri-instrument-features-and-caveats#MIRIInstrumentFeaturesandCaveats-glow_sticksGlowsticksintheMIRI4QPMcoronagraphs&gsc.tab=0), is visible in the images above. At mid-infrared wavelengths, MIRI is also highly sensitive to thermal emission from the telescope. To prevent faint sources from being obscured by scattered light and thermal emission, we perform background subtraction on the data before PSF subtraction.  </div>

SpaceKLIP's image manipulation tools class, `ImageTools`, offers two methods for background subtraction: a simple median subtraction using `subtract_background` and a more advanced technique `subtract_background_godoy` developed by Nico Godoy. 

* The `subtract_background` method performs a basic median subtraction but requires the removal of the first frame due to the reset switch charge decay (RSCD) in MIRI data, which can introduce nonlinearities at the start of integrations, non-linear ramps as the signal increases, latent images, and drifts in the slopes. Removal of the first frame can be done using `remove_frames`.

* In contrast, the `subtract_background_godoy` method refines background subtraction through an optimization process. Instead of simply subtracting the background, it analyzes specific sections of the image and adjusts the scaling to account for any residual background after the initial subtraction. This fine-tuning addresses subtle background variations, such as those caused by detector anomalies, without the need to remove the first frame.    



In [20]:
# Perform background subtraction to remove MIRI thermal background and glowstick. 
# This step is only required for MIRI.
imageTools.subtract_background_godoy(subdir='bgsub_godoy')

[spaceKLIP.imagetools:INFO] --> Concatenation JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
[spaceKLIP.imagetools:INFO]   --> Background subtraction: jw01386008001_04101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Background subtraction: jw01386009001_04101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Background subtraction: jw01386007001_04101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Background subtraction: jw01386007001_04101_00002_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Background subtraction: jw01386007001_04101_00003_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Background subtraction: jw01386007001_04101_00004_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Background subtraction: jw01386007001_04101_00005_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Background subtraction: jw01386007001_04101_00006_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Background subtraction

Let's examine the background subtracted results.

In [23]:
# Examine the background subtracted images.

spaceKLIP.plotting.display_image_comparisons(
    database,
    ['bpcleaned', 'bgsub_godoy'],  # Subdirectories to look for files in.
    restrict_to={'FILTER': filt,  # Sort by filter.
                 'TYPE': ['SCI', 'REF']  # Sort by file type SCI/REF.
    },
    interactive=True,  # Static or interactive plots?
    # vmin=0, vmax=500, # Define the min/max values for consistent image scaling.
    save_filename='plots_f1550c_backgroundsub.pdf')

IntSlider(value=0, description='Image Index:', max=10)

Output()

After background subtraction, the PSF becomes much more prominent in the images. Dark regions may also appear in the images, indicating areas where sources were present in the background frames and have been subtracted.

---

### Finish Pixel Cleanup

Optionally, any remaining bad pixels can be interpolated to replace NaNs with zeros.

In this case, this step is not strictly necessary since all the bad pixels have already been addressed in the previous steps. However, running this step for example purposes will not alter any pixel values, as they have already been fixed.

In [22]:
# Replace nans.
imageTools.replace_nans(cval=0., # Fill value.
                        types=['SCI', 'SCI_BG', 'REF', 'REF_BG'],
                        subdir='nanreplaced')

[spaceKLIP.imagetools:INFO] --> Concatenation JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
[spaceKLIP.imagetools:INFO]   --> Nan replacement: jw01386008001_04101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Nan replacement: replaced 0 nan pixel(s) with value 0.0 -- 0.00%
[spaceKLIP.imagetools:INFO]   --> Nan replacement: jw01386009001_04101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Nan replacement: replaced 0 nan pixel(s) with value 0.0 -- 0.00%
[spaceKLIP.imagetools:INFO]   --> Nan replacement: jw01386030001_02101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Nan replacement: replaced 0 nan pixel(s) with value 0.0 -- 0.00%
[spaceKLIP.imagetools:INFO]   --> Nan replacement: jw01386030001_03101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Nan replacement: replaced 0 nan pixel(s) with value 0.0 -- 0.00%
[spaceKLIP.imagetools:INFO]   --> Nan replacement: jw01386031001_02101_00001_mirimage_calints.fits
[spaceKLIP.im

---

### Align Frames

<div class="alert alert-info">
    
**MIRI-specific Information**: This step is not intuitive. Unlike with NIRCam, the MIRI PSF structure can change in structure and appearance. This can be seen as you flip through the science and reference images above. Attempting to align these frames will not work.  Therefore, we skip this step for MIRI.  </div>



---

### Pad Empty Space Around Frames

To give space to rotate and align during pyklip. This puts a region of NAN pixels around the outside. 

In [24]:
# Pad all frames.
imageTools.pad_frames(npix=80,
                      cval=np.nan,
                      types=['SCI', 'SCI_BG', 'REF', 'REF_BG'],
                      subdir='padded')

[spaceKLIP.imagetools:INFO] --> Concatenation JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
[spaceKLIP.imagetools:INFO]   --> Frame padding: jw01386008001_04101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Frame padding: old shape = (210, 215), new shape = (370, 375), fill value = nan
[spaceKLIP.imagetools:INFO]   --> Frame padding: jw01386009001_04101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Frame padding: old shape = (210, 215), new shape = (370, 375), fill value = nan
[spaceKLIP.imagetools:INFO]   --> Frame padding: jw01386030001_02101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Frame padding: old shape = (210, 215), new shape = (370, 375), fill value = nan
[spaceKLIP.imagetools:INFO]   --> Frame padding: jw01386030001_03101_00001_mirimage_calints.fits
[spaceKLIP.imagetools:INFO]   --> Frame padding: old shape = (210, 215), new shape = (370, 375), fill value = nan
[spaceKLIP.imagetools:INFO]   --> Frame padding: jw013860310

---

### Display the Cleaned Datasets 

After applying all the previous steps, review the cleaned data. If you notice artifacts near bad pixels, it is likely that some were not identified and have persisted. To address this, you may need to revisit the bad pixel identification process, adjust the parameters, or add a custom bad pixel map.


In [25]:
spaceKLIP.plotting.display_coron_dataset(
    database,
    restrict_to={'FILTER': filt,  # Sort by filter.
                 'TYPE': ['SCI']  # Sort by file type SCI/REF.
    },
    interactive=True,  # Set to False for static plots.
    zoom_center=3,  # Optional zoom factor; set to None to disable.
    bbox_color=None,  # Remove background text boxes.
    save_filename='plots_f1550c_stage2_cleaned.pdf'  # Save plots to PDF.
)

IntSlider(value=0, description='Image  Index:', max=1)

Output()

---

## Stage 3 Reductions: KLIP

### PSF Subtraction: Option Using pyKLIP

Now that we have cleaned up our data to account for alignment and bad pixels, we are ready to perform PSF subtraction. SpaceKLIP supports several algorithms for this step, including the recommended [`pyKLIP`](https://pyklip.readthedocs.io/en/latest/index.html) and the JWST [`Coron3Pipeline`](https://jwst-pipeline.readthedocs.io/en/latest/api/jwst.pipeline.Coron3Pipeline.html). In this case, we will use pyKLIP for subtraction.

You can customize the PSF subtraction process by adjusting the following settings:

> * `mode`: Choose from different image processing techniques, such as ADI, RDI, or a combination of both.
> * `annuli`: This parameter determines the number of concentric ring-shaped regions used for PSF subtraction. By specifying different numbers of annuli, you control the radial regions where the subtraction is applied, helping to remove the PSF at various distances from the center of the star. Default is [1].
> * `subsections`: This parameter defines the number of smaller regions or segments within each annulus where the PSF subtraction is applied. By breaking down the annuli into subsections, the algorithm can fine-tune the subtraction process. The default value is [1].
> * Number of KL Modes (`numbasis`): This parameter specifies how many Principal Component Analysis (PCA) modes are used to build the PSF model for subtraction. PCA modes capture different patterns and features in the data, allowing the model to represent and subtract the PSF more accurately. The default values are [1, 2, 5, 10, 20, 50, 100], which allows us to test different PSF models.
> * `algo`: Select the processing algorithm (here, klip).
> * `save_rolls`: A roll refers to a set of images taken during a specific pointing direction or orientation of the telescope. Enabling this parameter will save the PSF-subtracted versions of each individual science roll separately, in addition to the roll-combined final product.







These options help tailor the analysis to better suit your specific data and objectives.

In [26]:
# Run pyKLIP pipeline. Additional parameters for klip_dataset function can
# be passed using kwargs parameter.
spaceKLIP.pyklippipeline.run_obs(database=database,
                       kwargs={'mode': ['ADI', 'RDI', 'ADI+RDI'],
                               'annuli': [1],
                               'subsections': [1],
                               'numbasis': [1, 2, 5, 10, 20, 50],
                               'algo': 'klip',
                               'save_rolls': True},
                       subdir='klipsub')

[spaceKLIP.pyklippipeline:INFO] --> Concatenation JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
[spaceKLIP.pyklippipeline:INFO]   --> pyKLIP: mode = ADI, annuli = 1, subsections = 1
Begin align and scale images for each wavelength
Wavelength 1.552e-05 with index 0 has finished align and scale. Queuing for KLIP
Total number of tasks for KLIP processing is 1


  0%|          | 0/1 [00:00<?, ?it/s]

Closing threadpool
Derotating Images...
Writing Images to directory /System/Volumes/Data/user/kglidic/spaceKLIP/docs/source/tutorials/data_miri_hd65426/klipsub
wavelength collapsing reduced data of shape (b, N, wv, y, x):(6, 120, 1, 370, 375)
[spaceKLIP.pyklippipeline:INFO]   --> pyKLIP: mode = RDI, annuli = 1, subsections = 1
Begin align and scale images for each wavelength
Wavelength 1.552e-05 with index 0 has finished align and scale. Queuing for KLIP
Total number of tasks for KLIP processing is 1


  0%|          | 0/1 [00:00<?, ?it/s]

Closing threadpool
Derotating Images...
Writing Images to directory /System/Volumes/Data/user/kglidic/spaceKLIP/docs/source/tutorials/data_miri_hd65426/klipsub
wavelength collapsing reduced data of shape (b, N, wv, y, x):(6, 120, 1, 370, 375)
[spaceKLIP.pyklippipeline:INFO]   --> pyKLIP: mode = ADI+RDI, annuli = 1, subsections = 1
Begin align and scale images for each wavelength
Wavelength 1.552e-05 with index 0 has finished align and scale. Queuing for KLIP
Total number of tasks for KLIP processing is 1


  0%|          | 0/1 [00:00<?, ?it/s]

Closing threadpool
Derotating Images...
Writing Images to directory /System/Volumes/Data/user/kglidic/spaceKLIP/docs/source/tutorials/data_miri_hd65426/klipsub
wavelength collapsing reduced data of shape (b, N, wv, y, x):(6, 120, 1, 370, 375)
[spaceKLIP.database:INFO] --> Identified 1 concatenation(s)
[spaceKLIP.database:INFO]   --> Concatenation 1: JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
 TYPE  EXP_TYPE DATAMODL TELESCOP ... SUBSECTS    KLMODES     BUNIT  BLURFWHM
------ -------- -------- -------- ... -------- -------------- ------ --------
PYKLIP MIR_4QPM   STAGE3     JWST ...        1 1,2,5,10,20,50 MJy/sr      nan
PYKLIP MIR_4QPM   STAGE3     JWST ...        1 1,2,5,10,20,50 MJy/sr      nan
PYKLIP MIR_4QPM   STAGE3     JWST ...        1 1,2,5,10,20,50 MJy/sr      nan


The stage 3 information in the database is added to another table. The stage 2 information remains in the database, which is needed to maintain the information on rolls and references used in the reduction for forward modeling. In fact the stage 3 outputs include a JSON file that includes the table of the stage 2 data, so if you read in the stage 3 outputs it also learns about the stage 2 inputs.

In [27]:
database.summarize()

MIRI_F1550C_4QPM
	STAGE2: 15 files;	2 SCI, 9 REF, 4 BG
	STAGE3: 3 files;	3 PYKLIP


----

### Optional: Re-read Stage 3 Outputs into Database 
This shows how you can start re-analyses at this point, once you have run the previous steps.

Note, to read in stage 3 data you must set the `readlevel` parameter to 3. This invokes code for reading the stage-3 formatted data products, and also implicitly reads in the metadata about the stage 2 files used as input to stage 3. 

In [28]:
database = spaceKLIP.database.create_database(
                                    input_dir=os.path.join(data_root, 'klipsub'),
                                    file_type='*KLmodes-all.fits',
                                    output_dir=data_root,
                                    readlevel=3,
                                    pid=1386)

[spaceKLIP.database:INFO] --> Identified 1 concatenation(s)
[spaceKLIP.database:INFO]   --> Concatenation 1: JWST_MIRI_MIRIMAGE_F1550C_NONE_4QPM_1550_MASK1550
 TYPE  EXP_TYPE DATAMODL TELESCOP ... SUBSECTS    KLMODES     BUNIT  BLURFWHM
------ -------- -------- -------- ... -------- -------------- ------ --------
PYKLIP MIR_4QPM   STAGE3     JWST ...        1 1,2,5,10,20,50 MJy/sr      nan
PYKLIP MIR_4QPM   STAGE3     JWST ...        1 1,2,5,10,20,50 MJy/sr      nan
PYKLIP MIR_4QPM   STAGE3     JWST ...        1 1,2,5,10,20,50 MJy/sr      nan


In [29]:
database.summarize()

MIRI_F1550C_4QPM
	STAGE2: 15 files;	2 SCI, 9 REF, 4 BG
	STAGE3: 3 files;	3 PYKLIP


---

In [30]:
spaceKLIP.plotting.display_coron_dataset(
    database,
    stage3=True,
    restrict_to=filt,  # Sort by filter.
    interactive=True,  # Static or interactive plots?
    zoom_center=5,  # Optional zoom factor; set to None to disable.
    bbox_color=None,  # Remove background text boxes.
    save_filename='plots_f1550c_pyklip.pdf'  # Save plots to PDF.
)

IntSlider(value=0, description='Image  Index:', max=2)

Output()

Hurray! We’ve reached the end of the reduction process for MIRI coronagraphic data. Now, we can move on to Part 2 of this tutorial to analyze the results. At this stage, the planet should appear to the lower left of the KLIP center (indicated by the orange point), along with other background sources. To get a closer look at the planet, consider increasing the `zoom_center` factor in the plot function above.